In [1]:
import sys
import numpy as np
import pandas as pd
import random
import torch
from deepod.models.time_series import (
    COUTA, TimesNet, DeepSVDDTS, TranAD, 
    AnomalyTransformer, USAD, GDeepSVDDTS,DeepIsolationForestTS
)
from deepod.metrics import ts_metrics, point_adjustment
import warnings
warnings.filterwarnings("ignore")

In [2]:
def load_data(dataset):
    if dataset == 0:
        train_data = np.load('./data/SWaT/train.npy')
        test_data = np.load('./data/SWaT/test.npy')
        labels = np.load('./data/SWaT/labels.npy')
    elif dataset == 1:
        train_data = np.load('./data/SMAP/P-1_train.npy')
        test_data = np.load('./data/SMAP/P-1_test.npy')
        labels = np.load('./data/SMAP/P-1_labels.npy')[:, 0]
    elif dataset == 2:
        train_data = np.load('./data/MSL/C-1_train.npy')
        test_data = np.load('./data/MSL/C-1_test.npy')
        labels = np.load('./data/MSL/C-1_labels.npy')[:, 0]
    return train_data, test_data, labels

# Function to set random state for reproducibility
def set_random_state(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [3]:
# List of models to evaluate
models = {
    "DeepSVDDTS": lambda seed: DeepSVDDTS(network='TCN',random_state=seed),
    "USAD": lambda seed: USAD(random_state=seed),
    "DeepIsolationForestTS":lambda seed: DeepIsolationForestTS(random_state=seed),
    "TranAD": lambda seed: TranAD(random_state=seed),
    "COUTA": lambda seed: COUTA(random_state=seed),
    "AnomalyTransformer": lambda seed: AnomalyTransformer(random_state=seed),
    "GDeepSVDDTS": lambda seed: GDeepSVDDTS(epochs=70,random_state=seed)
}
results_table = []

In [4]:
for dataset_idx in range(3):
    train_data, test_data, labels = load_data(dataset_idx)
    print(f"Dataset {dataset_idx}: Running experiments...")
    for model_name, model_factory in models.items():
        print(f"Running model {model_name}...")
        adj_metrics_list = []
        
        for run in range(10):
            seed = 42 + dataset_idx * 10 + run  # Seed for reproducibility
            set_random_state(seed)
            print(f"Run {run + 1}/10 for {model_name} with Seed {seed}...")
            model = model_factory(seed)
            model.fit(train_data)
            scores = model.decision_function(test_data)
            adjusted_metrics = ts_metrics(labels, point_adjustment(labels, scores))
            adj_metrics_list.append([
                adjusted_metrics[0],  # Adjusted AUC
                adjusted_metrics[2],  # Adjusted F1
                adjusted_metrics[3],  # Adjusted Precision
                adjusted_metrics[4],  # Adjusted Recall
            ])


        adj_metrics_array = np.array(adj_metrics_list)
        adj_means, adj_stds = np.mean(adj_metrics_array, axis=0), np.std(adj_metrics_array, axis=0)

        # Append the results for table format
        results_table.append([
            f"Dataset {dataset_idx}",
            model_name,
            f"{adj_means[0]:.3f} ± {adj_stds[0]:.3f}",  # Adjusted AUC
            f"{adj_means[1]:.3f} ± {adj_stds[1]:.3f}",  # Adjusted F1
            f"{adj_means[2]:.3f} ± {adj_stds[2]:.3f}",  # Adjusted Precision
            f"{adj_means[3]:.3f} ± {adj_stds[3]:.3f}",  # Adjusted Recall
        ])


Dataset 0: Running experiments...
Running model DeepSVDDTS...
Run 1/10 for DeepSVDDTS with Seed 42...
Start Training...
ensemble size: 1
{'n_features': 1, 'n_hidden': '512', 'n_output': 64, 'activation': 'GELU', 'bias': False}
TCNnet(
  (network): Sequential(
    (0): TcnResidualBlock(
      (conv1): Conv1d(1, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp1): Chomp1d()
      (act1): GELU(approximate='none')
      (dropout1): Dropout(p=0.2, inplace=False)
      (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp2): Chomp1d()
      (act2): GELU(approximate='none')
      (dropout2): Dropout(p=0.2, inplace=False)
      (net): Sequential(
        (0): Conv1d(1, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (1): Chomp1d()
        (2): GELU(approximate='none')
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (5

testing: 100%|████████████████████████████████| 78/78 [00:00<00:00, 2665.18it/s]

Run 2/10 for DeepSVDDTS with Seed 43...
Start Training...
ensemble size: 1
{'n_features': 1, 'n_hidden': '512', 'n_output': 64, 'activation': 'GELU', 'bias': False}
TCNnet(
  (network): Sequential(
    (0): TcnResidualBlock(
      (conv1): Conv1d(1, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp1): Chomp1d()
      (act1): GELU(approximate='none')
      (dropout1): Dropout(p=0.2, inplace=False)
      (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp2): Chomp1d()
      (act2): GELU(approximate='none')
      (dropout2): Dropout(p=0.2, inplace=False)
      (net): Sequential(
        (0): Conv1d(1, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (1): Chomp1d()
        (2): GELU(approximate='none')
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (5): Chomp1d()
        (6): GELU(approximate='none')
        (7)

epoch 30, training loss: 0.003997, time: 0.0s
epoch 40, training loss: 0.003328, time: 0.0s
epoch 50, training loss: 0.002981, time: 0.0s
epoch 60, training loss: 0.002511, time: 0.0s
epoch 70, training loss: 0.002326, time: 0.0s
epoch 80, training loss: 0.002109, time: 0.0s
epoch 90, training loss: 0.002063, time: 0.0s
epoch100, training loss: 0.001951, time: 0.0s
Start Inference on the training data...


testing: 100%|████████████████████████████████| 78/78 [00:00<00:00, 3070.47it/s]

...

Run 9/10 for GDeepSVDDTS with Seed 70...
Start Training...
ensemble size: 1
TcnAE(
  (l1): Linear(in_features=512, out_features=32, bias=False)
  (encoder): Sequential(
    (0): TcnResidualBlock(
      (conv1): Conv1d(55, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp1): Chomp1d()
      (act1): GELU(approximate='none')
      (dropout1): Dropout(p=0.2, inplace=False)
      (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp2): Chomp1d()
      (act2): GELU(approximate='none')
      (dropout2): Dropout(p=0.2, inplace=False)
      (net): Sequential(
        (0): Conv1d(55, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (1): Chomp1d()
        (2): GELU(approximate='none')
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (5): Chomp1d()
        (6): GELU(approximate='none')
        (7): Dropout(p=0.2, inplace=Fa

epoch  1, training loss: 33.906725, time: 0.1s
epoch 10, training loss: 33.242354, time: 0.1s
epoch 20, training loss: 32.470815, time: 0.1s
epoch 30, training loss: 32.228316, time: 0.2s
epoch 40, training loss: 32.058640, time: 0.1s
epoch 50, training loss: 32.011065, time: 0.1s
epoch 60, training loss: 31.999504, time: 0.2s
epoch 70, training loss: 31.984962, time: 0.1s
Start Inference on the training data...


testing: 100%|█████████████████████████████████| 35/35 [00:00<00:00, 861.84it/s]


Run 10/10 for GDeepSVDDTS with Seed 71...
Start Training...
ensemble size: 1
TcnAE(
  (l1): Linear(in_features=512, out_features=32, bias=False)
  (encoder): Sequential(
    (0): TcnResidualBlock(
      (conv1): Conv1d(55, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp1): Chomp1d()
      (act1): GELU(approximate='none')
      (dropout1): Dropout(p=0.2, inplace=False)
      (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp2): Chomp1d()
      (act2): GELU(approximate='none')
      (dropout2): Dropout(p=0.2, inplace=False)
      (net): Sequential(
        (0): Conv1d(55, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (1): Chomp1d()
        (2): GELU(approximate='none')
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (5): Chomp1d()
        (6): GELU(approximate='none')
        (7): Dropout(p=0.2, inplace=F

testing: 100%|█████████████████████████████████| 35/35 [00:00<00:00, 674.17it/s]


In [12]:
from tabulate import tabulate
columns = [
    "Dataset", "Model", 
    "Adjusted AUC", "Adjusted F1", 
    "Adjusted Precision", "Adjusted Recall"
]
df_results = pd.DataFrame(results_table, columns=columns)

# Save results to a CSV file
df_results.to_csv("time_series_anomaly_metrics_summary_deep.csv", index=False)
print(tabulate(results_table, headers=columns))

Dataset    Model                  Adjusted AUC    Adjusted F1    Adjusted Precision    Adjusted Recall
---------  ---------------------  --------------  -------------  --------------------  -----------------
Dataset 0  DeepSVDDTS             0.924 ± 0.011   0.815 ± 0.001  0.986 ± 0.036         0.696 ± 0.024
Dataset 0  USAD                   0.804 ± 0.011   0.706 ± 0.002  0.715 ± 0.004         0.697 ± 0.000
Dataset 0  DeepIsolationForestTS  0.845 ± 0.005   0.778 ± 0.006  0.880 ± 0.014         0.697 ± 0.000
Dataset 0  TranAD                 0.865 ± 0.004   0.814 ± 0.001  0.990 ± 0.007         0.692 ± 0.004
Dataset 0  COUTA                  0.883 ± 0.041   0.814 ± 0.002  0.990 ± 0.013         0.692 ± 0.007
Dataset 0  AnomalyTransformer     0.914 ± 0.014   0.780 ± 0.034  0.915 ± 0.072         0.682 ± 0.029
Dataset 0  GDeepSVDDTS            0.868 ± 0.034   0.820 ± 0.014  0.991 ± 0.027         0.701 ± 0.039
Dataset 1  DeepSVDDTS             0.992 ± 0.001   0.925 ± 0.017  0.862 ± 0.029       